In [1]:
#package imports
from __future__ import division

import numpy as np
import datetime
import os
from wrf import to_np, getvar, get_basemap, latlon_coords
from netCDF4 import Dataset
import pandas as pd
from datetime import timedelta
from itertools import product
import math
import xarray as xr
from numpy.lib.stride_tricks import as_strided


In [2]:
#computation choosing

#wrf_file_folder_choice = 'wrf4km_sst_gombasin_cool_1'
#wrf_file_folder_choice = 'wrf4km_sst_gombasin_cool_2'
#wrf_file_folder_choice = 'wrf4km_sst_gomplume_cool_1'
#wrf_file_folder_choice = 'wrf4km_sst_gomplume_cool_2'
#wrf_file_folder_choice = 'wrf4km_sst_muregion_cool_1'
#wrf_file_folder_choice = 'wrf4km_sst_muregion_cool_2'
#wrf_file_folder_choice = 'wrf4km_sst_gombasin_warm_1'
#wrf_file_folder_choice = 'wrf4km_sst_gombasin_warm_2'
#wrf_file_folder_choice = 'wrf4km_sst_gomplume_warm_1'
#wrf_file_folder_choice = 'wrf4km_sst_gomplume_warm_2'
#wrf_file_folder_choice = 'wrf4km_sst_muregion_warm_1'
wrf_file_folder_choice = 'wrf4km_sst_muregion_warm_2'

choice_res = np.array([80])[0]

UH_thresholds = np.array([100,100,100,80,80,80,60,60,60,40,40,40])

dbz_thresholds = np.array([50,40,30,50,40,30,50,40,30,50,40,30])


In [6]:
for UH_threshold, dbz_threshold in zip(UH_thresholds, dbz_thresholds):

    file2_wrf_d02 = '/glade/scratch/molina/WRF_HYSPLIT_proj/'+wrf_file_folder_choice+'/wrfout_d01_2017-01-15_09:00:00'
    ncfile2_wrf_d02 = Dataset(file2_wrf_d02)
    while True:
        dbz2_wrf_d02 = np.array([0])
        if len(dbz2_wrf_d02) == 1:
            try: 
                dbz2_wrf_d02 = getvar(ncfile2_wrf_d02, "dbz")
            except ValueError:
                continue
        if len(dbz2_wrf_d02) != 1:
            print('File Opened...')
            break
    lats4, lons4 = latlon_coords(dbz2_wrf_d02)
    m = get_basemap(dbz2_wrf_d02)
    xMax, yMax = m(m.urcrnrlon, m.urcrnrlat) 
    xMin, yMin = m(m.llcrnrlon, m.llcrnrlat) 
    x_range = (xMax-xMin) / 1000 
    y_range = (yMax-yMin) / 1000 
    
    numXGrids = round(x_range / choice_res + .5,0) 
    numYGrids = round(y_range / choice_res + .5,0)
    xi = np.linspace(xMin, xMax, int(numXGrids))
    yi = np.linspace(yMin, yMax, int(numYGrids))
    
    
    #########################################################################################
    #########################################################################################
    #########################################################################################


    #create the range of dates to loop thru wrf files
    begdate = datetime.datetime.strptime('2017012109',"%Y%m%d%H") 
    enddate = datetime.datetime.strptime('2017012309',"%Y%m%d%H") 
    monthstr = datetime.datetime.strftime(begdate,"%B")
    #format into strings
    bmonth = '%02d' % begdate.month
    bday = '%02d' % begdate.day
    bbday = begdate.day
    #loop thru hours
    dates = []
    while begdate < enddate:
      dates.append(begdate)
      begdate+=datetime.timedelta(hours=1) 

    uvv_coarse = np.zeros((len(dates),yi.shape[0]-1, xi.shape[0]-1))

    counter = 0
    

    for count, dt in enumerate(dates):

      YY = dt.year

      MM = "%02d"%dt.month
      DD = "%02d"%dt.day
      HH = "%02d"%dt.hour

      HH_current = dt.hour

      if count != 0:
          if HH_current != HH_previous:    
              counter += 1

      wrffile = '/glade/scratch/molina/WRF_HYSPLIT_proj/'+wrf_file_folder_choice+'/wrfout_d01_2017-'+MM+'-'+DD+'_'+HH+':00:00'

      if os.path.isfile(wrffile)==True:

        ncfile = Dataset(wrffile)

        uvv = getvar(ncfile, "updraft_helicity", meta=False)
        uvv_val = uvv

        uvv2 = xr.open_dataset(wrffile, decode_cf=True).UP_HELI_MAX.values[0,:,:]
        uvv2_val = uvv2

        dbz = getvar(ncfile, "dbz", meta=False)
        dbz_val = dbz[0,:,:]

        for i, j in product(range(0,uvv2.shape[0]), range(0,uvv2.shape[1])):

            max_UH = np.nanmax([uvv_val[i,j],uvv2_val[i,j]])

            if max_UH >= UH_threshold and dbz_val[i,j] >= dbz_threshold:

                latsnow, lonsnow = to_np(lats4)[i,j], to_np(lons4)[i,j]
                x_proj,y_proj = m(lonsnow, latsnow)
                grid, _, _ = np.histogram2d(np.array([y_proj]),np.array([x_proj]), bins=[yi,xi])

                uvv_coarse[counter,:,:] += grid > 0

      else:

        print("file doesn't exist", os.path.basename(ncfile))

      HH_previous = dt.hour

      print(str(dt)+' complete...')

    print(f'UH ({UH_threshold}) and DBZ ({dbz_threshold}) finished.')

    x1, y1 = xi, yi

    x2 = np.zeros(x1.shape)
    y2 = np.zeros(y1.shape)
    x3 = np.zeros(x1.shape)
    y3 = np.zeros(y1.shape)

    for i, j in product(range(len(x1)),range(len(y1))):

        x2[i], y2[j] = m(x1[i], y1[j], inverse=True)

        x2[i] = x2[i]-0.4
        y2[j] = y2[j]-0.4

        x3[i], y3[j] = m(x2[i], y2[j])

    x4, y4 = x3[1:], y3[1:]

    uvv_coarse1 = np.nansum(uvv_coarse, axis=0)


    ###############################################################################
    ###############################################################################
    ###############################################################################


    final_filed = xr.Dataset({'UHdbz':(['y','x'], uvv_coarse1),
                              'UHdbz_dates':(['time','y','x'], uvv_coarse),                              
                              },

                              coords={'lon':(['x'],x4),
                                      'lat':(['y'],y4),
                                      'time':(['time'],pd.to_datetime(dates))},

                              attrs={'File Author':'Maria J. Molina'})

    
    final_filed.to_netcdf('/glade/scratch/molina/WRF_HYSPLIT_proj/'+wrf_file_folder_choice+'/ver_grid_80_UH_'+str(UH_threshold)+'_Z_'+str(dbz_threshold)+'.nc')


###############################################################################
###############################################################################
###############################################################################


File Opened...


KeyboardInterrupt: 